In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
path = "/content/gdrive/My Drive/Independent"
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import tensorflow as tf
from scipy import sparse
from keras import backend as K
from keras.models import Sequential
from sklearn.utils import class_weight
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def save(users_posts, users_label):
    sparse.save_npz("./result/v_countposts.npz", users_posts, False)

    file = open("./result/v_label.txt", "w")
    for user_label in users_label:
        file.write(str(user_label) + "\n")
    file.close()

def read_list(path):
    result = []
    with open(path, "r") as f:
        for line in f:
          temp = line.strip("\n")
          result.append(int(temp))
    return result

def read_npz(path):
    sparse_matrix = sparse.load_npz(path)
    return sparse_matrix

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [ ]:
# load dataset
X= read_npz("./result/t_tfidf_posts.npz")
y= read_list("./result/t_label.txt")

scaler = StandardScaler()
X = X.toarray()
X = X[:, 50000:] # 0-50000为乱码
X = scaler.fit_transform(X)
n, d= X.shape

precision_list = []
recall_list = []
f1_list = []
accuracy_list = []

for i in range(1,10):
  # create model
  model = Sequential()
  model.add(Dense(d, input_dim=d, activation='relu', kernel_initializer='random_normal'))
  model.add(Dropout(0.1))
  model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
  model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['acc',f1_m,precision_m,recall_m])

  K.get_session().run(tf.local_variables_initializer())

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

  # train model
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=i)
  history = model.fit(X_train, y_train, batch_size=40, epochs=10, verbose =0)

  # evaluate the model
  loss, accuracy, f1, precision, recall = model.evaluate(X, y, verbose=0)
  precision_list.append(precision)
  recall_list.append(recall)
  f1_list.append(f1)
  accuracy_list.append(accuracy)

print(np.mean(precision_list) , " " , np.std(precision_list))
print(np.mean(recall_list) , " " , np.std(recall_list))
print(np.mean(f1_list) , " " , np.std(f1_list))
print(np.mean(accuracy_list), " ", np.std(accuracy_list))

0.6964202920595804   0.0873192859800281
0.7350115842289395   0.07006654720636513
0.6872495611508688   0.06853493677450373
0.9595555663108826   0.01307339182893611
